# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
def mask_test_edges_biased_negative_samples(adj, test_percent=10.):
    """
    Task 2: General Biased Negative Samples (B.N.S.) Directed Link
    Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Retrieve all unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj_tilde)

    # Number of positive (= to number of negative) test node pairs to sample
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))

    # Sampling of positive node pairs
    edges_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_idx)
    test_edges_idx = edges_idx[:num_test]
    test_edges = edges_positive[test_edges_idx]

    # In this setting, the reverse node pairs constitute negative samples
    test_edges_false = np.fliplr(test_edges)

    # Get training incomplete adjacency matrix
    train_edges = np.delete(edges_positive, np.hstack([test_edges_idx]), axis = 0)
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    # Validation set: not implemented for Task 2
    # therefore, val_edges and val_edges_false are None
    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0], replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [6]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(task=2)

# 定义模型

In [7]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [8]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

context_embedding_dim = 12
importance_embedding_dim = 48
beta = 4.8
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 29.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'Cornell'

In [9]:
all_best_auc = []
all_best_ap = []
for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    all_best_ap.append(auc_first_best_ap)
    all_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 127.8916
Epoch: 01, Loss: 127.7765
Epoch: 02, Loss: 127.5588
Epoch: 03, Loss: 127.2505
Epoch: 04, Loss: 126.8632
AP SCORE：  0.6416140533634906
AUC SCORE:  0.6160000000000001
Epoch: 00, Loss: 126.4073
Epoch: 01, Loss: 126.3877
Epoch: 02, Loss: 126.3506
Epoch: 03, Loss: 126.2980
Epoch: 04, Loss: 126.2318
AP SCORE：  0.6511055824415241
AUC SCORE:  0.6335999999999999
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 126.1540
Epoch: 01, Loss: 126.0468
Epoch: 02, Loss: 125.8439
Epoch: 03, Loss: 125.5560
Epoch: 04, Loss: 125.1933
AP SCORE：  0.6570667636204721
AUC SCORE:  0.6464
Epoch: 00, Loss: 124.7654
Epoch: 01, Loss: 124.7480
Epoch: 02, Loss: 124.7150
Epoch: 03, Loss: 124.6683
Epoch: 04, Loss: 124.6095
AP SCORE：  0.6599014260340819
AUC SCORE:  0.6527999999999999
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 124.5401
Epoch: 01, Loss: 124.4383
Epoch: 02,

Epoch: 03, Loss: 99.9123
Epoch: 04, Loss: 99.8501
AP SCORE：  0.7717402440145413
AUC SCORE:  0.8128000000000001
######################### 模块循环 ： 21 ##########################
Epoch: 00, Loss: 99.7762
Epoch: 01, Loss: 99.7041
Epoch: 02, Loss: 99.5675
Epoch: 03, Loss: 99.3731
Epoch: 04, Loss: 99.1275
AP SCORE：  0.7729094747837721
AUC SCORE:  0.8144
Epoch: 00, Loss: 98.8366
Epoch: 01, Loss: 98.8180
Epoch: 02, Loss: 98.7826
Epoch: 03, Loss: 98.7323
Epoch: 04, Loss: 98.6686
AP SCORE：  0.7758380462123435
AUC SCORE:  0.8176
######################### 模块循环 ： 22 ##########################
Epoch: 00, Loss: 98.5929
Epoch: 01, Loss: 98.5220
Epoch: 02, Loss: 98.3874
Epoch: 03, Loss: 98.1960
Epoch: 04, Loss: 97.9542
AP SCORE：  0.7779886424192058
AUC SCORE:  0.8207999999999999
Epoch: 00, Loss: 97.6678
Epoch: 01, Loss: 97.6487
Epoch: 02, Loss: 97.6126
Epoch: 03, Loss: 97.5611
Epoch: 04, Loss: 97.4959
AP SCORE：  0.7779886424192058
AUC SCORE:  0.8207999999999999
######################### 模块循环 ： 23 #######

Epoch: 02, Loss: 76.7022
Epoch: 03, Loss: 76.6520
Epoch: 04, Loss: 76.5889
AP SCORE：  0.8412322715963393
AUC SCORE:  0.8704
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 76.5144
Epoch: 01, Loss: 76.4661
Epoch: 02, Loss: 76.3744
Epoch: 03, Loss: 76.2441
Epoch: 04, Loss: 76.0795
AP SCORE：  0.8412322715963393
AUC SCORE:  0.8704
Epoch: 00, Loss: 75.8846
Epoch: 01, Loss: 75.8664
Epoch: 02, Loss: 75.8320
Epoch: 03, Loss: 75.7833
Epoch: 04, Loss: 75.7219
AP SCORE：  0.8412322715963393
AUC SCORE:  0.8704
######################### 模块循环 ： 44 ##########################
Epoch: 00, Loss: 75.6497
Epoch: 01, Loss: 75.6023
Epoch: 02, Loss: 75.5123
Epoch: 03, Loss: 75.3845
Epoch: 04, Loss: 75.2231
AP SCORE：  0.8412322715963393
AUC SCORE:  0.8704
Epoch: 00, Loss: 75.0319
Epoch: 01, Loss: 75.0142
Epoch: 02, Loss: 74.9808
Epoch: 03, Loss: 74.9335
Epoch: 04, Loss: 74.8739
AP SCORE：  0.8423652765224476
AUC SCORE:  0.872
######################### 模块循环 ： 45 ###################

Epoch: 03, Loss: 61.2718
Epoch: 04, Loss: 61.2426
AP SCORE：  0.8881065578172792
AUC SCORE:  0.9168000000000001
######################### 模块循环 ： 65 ##########################
Epoch: 00, Loss: 61.2082
Epoch: 01, Loss: 61.1752
Epoch: 02, Loss: 61.1125
Epoch: 03, Loss: 61.0233
Epoch: 04, Loss: 60.9106
AP SCORE：  0.8881065578172792
AUC SCORE:  0.9168000000000001
Epoch: 00, Loss: 60.7768
Epoch: 01, Loss: 60.7684
Epoch: 02, Loss: 60.7525
Epoch: 03, Loss: 60.7299
Epoch: 04, Loss: 60.7014
AP SCORE：  0.8896875854852635
AUC SCORE:  0.9184
######################### 模块循环 ： 66 ##########################
Epoch: 00, Loss: 60.6678
Epoch: 01, Loss: 60.6352
Epoch: 02, Loss: 60.5735
Epoch: 03, Loss: 60.4856
Epoch: 04, Loss: 60.3744
AP SCORE：  0.8896875854852635
AUC SCORE:  0.9184
Epoch: 00, Loss: 60.2426
Epoch: 01, Loss: 60.2344
Epoch: 02, Loss: 60.2188
Epoch: 03, Loss: 60.1968
Epoch: 04, Loss: 60.1690
AP SCORE：  0.8923127728915603
AUC SCORE:  0.9216
######################### 模块循环 ： 67 ###################

AP SCORE：  0.9179296949860689
AUC SCORE:  0.9392
######################### 模块循环 ： 87 ##########################
Epoch: 00, Loss: 51.3640
Epoch: 01, Loss: 51.3421
Epoch: 02, Loss: 51.3007
Epoch: 03, Loss: 51.2418
Epoch: 04, Loss: 51.1673
AP SCORE：  0.9179296949860689
AUC SCORE:  0.9392
Epoch: 00, Loss: 51.0791
Epoch: 01, Loss: 51.0743
Epoch: 02, Loss: 51.0653
Epoch: 03, Loss: 51.0524
Epoch: 04, Loss: 51.0361
AP SCORE：  0.9179296949860689
AUC SCORE:  0.9392
######################### 模块循环 ： 88 ##########################
Epoch: 00, Loss: 51.0170
Epoch: 01, Loss: 50.9956
Epoch: 02, Loss: 50.9551
Epoch: 03, Loss: 50.8974
Epoch: 04, Loss: 50.8245
AP SCORE：  0.9199325809889547
AUC SCORE:  0.9392
Epoch: 00, Loss: 50.7381
Epoch: 01, Loss: 50.7334
Epoch: 02, Loss: 50.7246
Epoch: 03, Loss: 50.7121
Epoch: 04, Loss: 50.6963
AP SCORE：  0.9199325809889547
AUC SCORE:  0.9392
######################### 模块循环 ： 89 ##########################
Epoch: 00, Loss: 50.6777
Epoch: 01, Loss: 50.6567
Epoch: 02, Loss:

...

AP SCORE：  0.7147679507446816
AUC SCORE:  0.7136
Epoch: 00, Loss: 107.5493
Epoch: 01, Loss: 107.5366
Epoch: 02, Loss: 107.5125
Epoch: 03, Loss: 107.4781
Epoch: 04, Loss: 107.4346
AP SCORE：  0.7200284267028318
AUC SCORE:  0.7184
######################### 模块循环 ： 11 ##########################
Epoch: 00, Loss: 107.3830
Epoch: 01, Loss: 107.2957
Epoch: 02, Loss: 107.1301
Epoch: 03, Loss: 106.8948
Epoch: 04, Loss: 106.5979
AP SCORE：  0.724046787427075
AUC SCORE:  0.7232000000000001
Epoch: 00, Loss: 106.2467
Epoch: 01, Loss: 106.2336
Epoch: 02, Loss: 106.2087
Epoch: 03, Loss: 106.1732
Epoch: 04, Loss: 106.1283
AP SCORE：  0.7267030011320692
AUC SCORE:  0.7263999999999999
######################### 模块循环 ： 12 ##########################
Epoch: 00, Loss: 106.0750
Epoch: 01, Loss: 105.9898
Epoch: 02, Loss: 105.8284
Epoch: 03, Loss: 105.5990
Epoch: 04, Loss: 105.3095
AP SCORE：  0.7395305873389657
AUC SCORE:  0.7312
Epoch: 00, Loss: 104.9670
Epoch: 01, Loss: 104.9534
Epoch: 02, Loss: 104.9275
Epoch: 0

Epoch: 04, Loss: 83.9289
AP SCORE：  0.7916389553805164
AUC SCORE:  0.776
######################### 模块循环 ： 31 ##########################
Epoch: 00, Loss: 83.8441
Epoch: 01, Loss: 83.7865
Epoch: 02, Loss: 83.6775
Epoch: 03, Loss: 83.5226
Epoch: 04, Loss: 83.3272
AP SCORE：  0.7926971564387175
AUC SCORE:  0.7776000000000001
Epoch: 00, Loss: 83.0963
Epoch: 01, Loss: 83.0754
Epoch: 02, Loss: 83.0358
Epoch: 03, Loss: 82.9798
Epoch: 04, Loss: 82.9095
AP SCORE：  0.7935971866410734
AUC SCORE:  0.7776
######################### 模块循环 ： 32 ##########################
Epoch: 00, Loss: 82.8269
Epoch: 01, Loss: 82.7706
Epoch: 02, Loss: 82.6641
Epoch: 03, Loss: 82.5127
Epoch: 04, Loss: 82.3218
AP SCORE：  0.7984015851411239
AUC SCORE:  0.7824
Epoch: 00, Loss: 82.0962
Epoch: 01, Loss: 82.0759
Epoch: 02, Loss: 82.0374
Epoch: 03, Loss: 81.9831
Epoch: 04, Loss: 81.9148
AP SCORE：  0.7990622458017846
AUC SCORE:  0.784
######################### 模块循环 ： 33 ##########################
Epoch: 00, Loss: 81.8346
Epoch:

Epoch: 01, Loss: 67.6694
Epoch: 02, Loss: 67.6511
Epoch: 03, Loss: 67.6251
Epoch: 04, Loss: 67.5924
AP SCORE：  0.8243414096797865
AUC SCORE:  0.8064
######################### 模块循环 ： 52 ##########################
Epoch: 00, Loss: 67.5537
Epoch: 01, Loss: 67.5198
Epoch: 02, Loss: 67.4555
Epoch: 03, Loss: 67.3642
Epoch: 04, Loss: 67.2486
AP SCORE：  0.8193414096797864
AUC SCORE:  0.8048000000000001
Epoch: 00, Loss: 67.1118
Epoch: 01, Loss: 67.1023
Epoch: 02, Loss: 67.0844
Epoch: 03, Loss: 67.0590
Epoch: 04, Loss: 67.0269
AP SCORE：  0.8193414096797864
AUC SCORE:  0.8048000000000001
######################### 模块循环 ： 53 ##########################
Epoch: 00, Loss: 66.9889
Epoch: 01, Loss: 66.9557
Epoch: 02, Loss: 66.8927
Epoch: 03, Loss: 66.8030
Epoch: 04, Loss: 66.6895
AP SCORE：  0.817997536162
AUC SCORE:  0.8032
Epoch: 00, Loss: 66.5550
Epoch: 01, Loss: 66.5457
Epoch: 02, Loss: 66.5280
Epoch: 03, Loss: 66.5028
Epoch: 04, Loss: 66.4710
AP SCORE：  0.8193414096797864
AUC SCORE:  0.80480000000000

Epoch: 03, Loss: 57.9749
Epoch: 04, Loss: 57.9003
AP SCORE：  0.8092210584788528
AUC SCORE:  0.7936
Epoch: 00, Loss: 57.8118
Epoch: 01, Loss: 57.8074
Epoch: 02, Loss: 57.7992
Epoch: 03, Loss: 57.7875
Epoch: 04, Loss: 57.7728
AP SCORE：  0.8092210584788528
AUC SCORE:  0.7936
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 57.7554
Epoch: 01, Loss: 57.7339
Epoch: 02, Loss: 57.6931
Epoch: 03, Loss: 57.6352
Epoch: 04, Loss: 57.5619
AP SCORE：  0.8092210584788528
AUC SCORE:  0.7936
Epoch: 00, Loss: 57.4750
Epoch: 01, Loss: 57.4707
Epoch: 02, Loss: 57.4627
Epoch: 03, Loss: 57.4513
Epoch: 04, Loss: 57.4369
AP SCORE：  0.8092210584788528
AUC SCORE:  0.7936
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 57.4199
Epoch: 01, Loss: 57.3988
Epoch: 02, Loss: 57.3588
Epoch: 03, Loss: 57.3018
Epoch: 04, Loss: 57.2298
AP SCORE：  0.8092210584788528
AUC SCORE:  0.7936
Epoch: 00, Loss: 57.1445
Epoch: 01, Loss: 57.1403
Epoch: 02, Loss: 57.1324
Epoc

Epoch: 01, Loss: 52.0038
Epoch: 02, Loss: 51.9745
Epoch: 03, Loss: 51.9329
Epoch: 04, Loss: 51.8802
AP SCORE：  0.8162093479221357
AUC SCORE:  0.7984
Epoch: 00, Loss: 51.8178
Epoch: 01, Loss: 51.8152
Epoch: 02, Loss: 51.8102
Epoch: 03, Loss: 51.8032
Epoch: 04, Loss: 51.7944
AP SCORE：  0.8162093479221357
AUC SCORE:  0.7984
######################### 模块循环 ： 94 ##########################
Epoch: 00, Loss: 51.7840
Epoch: 01, Loss: 51.7688
Epoch: 02, Loss: 51.7400
Epoch: 03, Loss: 51.6991
Epoch: 04, Loss: 51.6473
AP SCORE：  0.8167378891483514
AUC SCORE:  0.7999999999999999
Epoch: 00, Loss: 51.5858
Epoch: 01, Loss: 51.5833
Epoch: 02, Loss: 51.5785
Epoch: 03, Loss: 51.5716
Epoch: 04, Loss: 51.5630
AP SCORE：  0.8167378891483514
AUC SCORE:  0.7999999999999999
######################### 模块循环 ： 95 ##########################
Epoch: 00, Loss: 51.5528
Epoch: 01, Loss: 51.5379
Epoch: 02, Loss: 51.5096
Epoch: 03, Loss: 51.4693
Epoch: 04, Loss: 51.4183
AP SCORE：  0.8167378891483514
AUC SCORE:  0.7999999999

In [10]:
print("MEAN AP : ", np.array(all_best_ap).mean())
print("STD AP : ", np.array(all_best_ap).std())
print("MEAN AUC : ", np.array(all_best_auc).mean())
print("STD AUC : ", np.array(all_best_auc).std())
"""
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)
"""

MEAN AP :  0.8506449279551905
STD AP :  0.054398585533123184
MEAN AUC :  0.8587200000000001
STD AUC :  0.05701502959746667


'\nprint("AUC FIRST BEST AUC: ", auc_first_best_auc)\nprint("AUC FIRST BEST AP: ", auc_first_best_ap)\nprint("AP FIRST BEST AUC: ", ap_first_best_auc)\nprint("AP FIRST BEST AP: ", ap_first_best_ap)\n'